In [1]:
import json
import PySAM.Windpower as wp
import pandas as pd

# USER INPUTS
# JSON file and weather file should be saved to the same folder as the ipynb file.
json_file_name = 'SAM_Wind_Model_Template_windpower.json'
generator_inputs_location = 'C:\\Users\\fwagg\\OneDrive\\Desktop\\Renewable Energy\\Project 2 - 100% Renewable SD\\SAM\\Wind_Generator_Input_Data.csv'
wind_weather_file_location = 'C:\\Users\\fwagg\\OneDrive\\Desktop\\Renewable Energy\\Project 2 - 100% Renewable SD\\SAM\\Wind_WeatherFile.csv'
output_data_location = 'C:\\Users\\fwagg\\OneDrive\\Desktop\\Renewable Energy\\Project 2 - 100% Renewable SD\\SAM\\Total_Wind_Output_Timeseries.csv'

# Create a new instance of a pySAM model
wind_model = wp.new()

# Declare all variables as strings. Spaces must be replaced with '+', i.e., change 'John Smith' to 'John+Smith'.
# Define the lat, long of the location and the year
api_key = 'zRso1hhdHGyzE1JXUAVz5AEkWM40yTr7yrXJOcDW'
attributes = 'pressure_100m, temperature_120m, winddirection_120m, windspeed_120m'
leap_year = 'false'
interval = '30'
utc = 'false'
your_name = 'Frank+Waggoner'
reason_for_use = 'School+Project'
your_affiliation = 'CU+Boulder'
your_email = 'frwa2461@colorado.edu'
mailing_list = 'false'
hubheight = '120'

# Get template inputs from the JSON file. File exported from SAM PV Model Template.sam
with open(json_file_name, 'r') as f:
        wind_inputs = json.load( f )
for k, v in wind_inputs.items():
    if k != 'number_inputs':
        wind_model.value(k, v)  

# Import generator data
wind_input_data = pd.read_csv(generator_inputs_location, index_col=None)
wind_capacities = wind_input_data.iloc[:,1].tolist()
wind_lats = wind_input_data.iloc[:,2].tolist()
wind_lons = wind_input_data.iloc[:,3].tolist()
wind_number_gens = len(wind_input_data.iloc[:,1])
wind_years = [2012]

# Create output data structure
wind_total_output = []
header_list = ['Year','Month','Day','Hour','Minute','Total Wind Output (kW)']

for i in range(1):
    # Change the value of iterative weather file variables
    lat = 32.72
    lon = -116.26
    #lat = wind_lats[i+1]
    #lon = wind_lons[i+1]
    year = wind_years[0]
    # Declare url string for weather file.
    srw = 'https://developer.nrel.gov/api/wind-toolkit/v2/wind/wtk-srw-download?api_key={api}&lat={lat}&lon={lon}&year={year}&hubheight={hubheight}'.format(year=year, lat=lat, lon=lon, api=api_key, hubheight=hubheight)
    df = []
    df = pd.read_csv(srw, index_col = False)
    # Create weather csv file
    df.to_csv(wind_weather_file_location, index = False)
    # Change the value of iterative wind system variables
    #wind_model.value('wind_resource_filename', 'C:\\Users\\fwagg\\OneDrive\\Desktop\\Renewable Energy\\Project 2 - 100% Renewable SD\\SAM\\Wind_WeatherFile.csv')
    #wind_model.value('system_capacity', 143000)
    #wind_model.desired_farm_size = 
    
    wind_model.Resource.wind_resource_filename = wind_weather_file_location
    wind_model.Farm.system_capacity = 143000
    
    # Run the module
    wind_model.execute()
    # Store power output data in a cumulative list
    for j in range(len(wind_model.Outputs.gen)):
        # Build empty total output list to store total output time series data
        # Add time series data to empty list
        if i == 0:
            wind_total_output.append([0,0,0,0,0,0])
            wind_total_output[j][0] = df.iat[j+2,0]
            wind_total_output[j][1] = df.iat[j+2,1]
            wind_total_output[j][2] = df.iat[j+2,2]
            wind_total_output[j][3] = df.iat[j+2,3]
            wind_total_output[j][4] = df.iat[j+2,4]
        # Fill in cumulative gen output time series data
        wind_total_output[j][5] = wind_total_output[j][5] + wind_model.Outputs.gen[j]
    # Print('Annual AC output for {capacity:,.2f} kW system is {output:,.0f} kWh.'.format(capacity = pv_model.value('system_capacity'), output = pv_model.Outputs.ac_annual) )
outputDF = pd.DataFrame(wind_total_output)
outputDF.to_csv(output_data_location, index=False, header=header_list)
print('Done')
print(wind_total_output)

Done
[['C', 'atm', 'm/s', 'Degrees', nan, 48376.64982504308], ['120', '120', '120', '120', nan, 55811.396611430304], ['14.5', '0.87', '8.300', '39.1', nan, 58667.89094725757], ['14.7', '0.87', '8.700', '36.6', nan, 45883.91817273274], ['14.5', '0.87', '8.850', '36.6', nan, 40732.301284899026], ['14.4', '0.87', '8.150', '38.0', nan, 42227.9291824748], ['14.4', '0.87', '7.840', '44.7', nan, 48044.27926541482], ['14.4', '0.87', '7.930', '41.2', nan, 47925.92064864019], ['14.3', '0.87', '8.280', '38.9', nan, 40278.90481334405], ['14.2', '0.87', '8.270', '43.6', nan, 43603.69534089545], ['14.3', '0.87', '7.810', '40.6', nan, 34251.21800685271], ['14.3', '0.87', '8.010', '36.0', nan, 33211.466144671955], ['14.5', '0.87', '7.450', '31.8', nan, 28024.687141672457], ['14.8', '0.87', '7.390', '36.5', nan, 29228.33727823355], ['15.2', '0.87', '6.950', '39.6', nan, 26895.835055197975], ['15.4', '0.87', '7.060', '41.0', nan, 32036.324627424536], ['15.6', '0.87', '6.850', '37.8', nan, 36417.39321260